Estudo baseado nos seguintes artigos:

* [Having an Imbalanced Dataset? Here Is How You Can Fix It.](https://towardsdatascience.com/having-an-imbalanced-dataset-here-is-how-you-can-solve-it-1640568947eb)

* [Dealing with unbalanced data in machine learning](https://www.r-bloggers.com/dealing-with-unbalanced-data-in-machine-learning/)

In [5]:
library(caret) # short for _C_lassification _A_nd _RE_gression _T_raining
# http://topepo.github.io/caret/index.html